<a href="https://colab.research.google.com/github/MS-H2020/Open/blob/main/Open_Images_Object_Detection_RVC_2020_edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Open Images Object Detection RVC 2020 edition](https://www.kaggle.com/competitions/open-images-object-detection-rvc-2020/code?competitionId=19205&sortBy=voteCount&excludeNonAccessedDatasources=true)

## Refference
1.   [Tensorflow Hub Faster Rcnn](https://www.kaggle.com/code/c7934597/tensorflow-hub-faster-rcnn)



## 0. Setting for Google Colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
INPUT_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020/01_input"
ANALYSIS_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020/03_code-analysis"
MODEL_DIR:str = "/content/drive/MyDrive/open-images-object-detection-rvc-2020//04_model"
SUBMISSION_DIR:str = '/content/drive/MyDrive/open-images-object-detection-rvc-2020/05_submission'
CHECKPOINT_PATH:str = MODEL_DIR

In [ ]:
import os, gc, glob, shutil, zipfile
shutil.unpack_archive(INPUT_DIR+'/test.zip', './')

## 1. Importing Modules

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from six import BytesIO
import numpy as np
import xml.etree.ElementTree as et
import ast
import tqdm
from itertools import chain
from xml.dom import minidom
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import cv2
import glob
import time

In [ ]:
path1 = INPUT_DIR + '/test/'

In [ ]:
df = pd.read_csv(INPUT_DIR + '/sample_submission.csv')
df.head()

In [ ]:
df.shape

In [ ]:
ids = []
for i in range(len(df)):
    ids.append(df['ImageId'][i])

In [ ]:
ids[0:5]

In [ ]:
img_data=[]
for i in range(len(df)):
    img_data.append(glob.glob('./test/{0}.jpg'.format(ids[i])))

In [ ]:
img_data[0:5]

In [ ]:
img_data = list(chain.from_iterable(img_data))

In [ ]:
img_data[0:5]

In [ ]:
def get_prediction_string(result):
    with tf.device('/device:GPU:0'):
        df = pd.DataFrame(columns=['Ymin','Xmin','Ymax', 'Xmax','Score','Label','Class_label','Class_name'])
        min_score=0.01
        for i in range(result['detection_boxes'].shape[0]):
           if (result["detection_scores"][i]) >= min_score:
              df.loc[i]= tuple(result['detection_boxes'][i])+(result["detection_scores"][i],)+(result["detection_class_labels"][i],)+(result["detection_class_names"][i],)+(result["detection_class_entities"][i],)
        return df

In [ ]:
import tensorflow.compat.v1
tensorflow.compat.v1.disable_v2_behavior()

### [TensorFlow, "Object Detection"](https://www.tensorflow.org/hub/tutorials/object_detection)

In [ ]:
module_handle = "https://www.kaggle.com/models/google/faster-rcnn-inception-resnet-v2/TensorFlow1/faster-rcnn-openimages-v4-inception-resnet-v2/1"
with tf.device('/device:GPU:0'):
    with tf.Graph().as_default():
        detector = hub.load(module_handle).signatures['default']
        image_string_placeholder = tf.placeholder(tf.string)
        decoded_image = tf.image.decode_jpeg(img, channels=3)
        decoded_image_float = tf.image.convert_image_dtype(
            image=decoded_image, dtype=tf.float32)
        module_input = tf.expand_dims(decoded_image_float, 0)
        result = detector(module_input, as_dict=True)
        init_ops = [tf.global_variables_initializer(), tf.tables_initializer()]

        session = tf.Session()
        session.run(init_ops)